# Heating Patterns Analysis

This notebook analyzes heating usage patterns across different rooms in the smart home.

## Analysis Goals
- Identify room-specific heating patterns and schedules
- Analyze temperature setpoints vs actual temperatures
- Calculate heating energy consumption by room and time
- Identify optimization opportunities for energy savings

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from influxdb_client import InfluxDBClient
from datetime import datetime, timedelta
import pytz
from scipy import signal
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Database Connection

In [ ]:
# InfluxDB connection parameters
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "your-token-here"
INFLUX_ORG = "loxone"
INFLUX_BUCKET = "loxone"

# Initialize InfluxDB client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
query_api = client.query_api()

## 3. Data Loading Functions

In [ ]:
def load_temperature_data(start_date, end_date):
    """Load room temperature data from InfluxDB"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "temperature")
        |> filter(fn: (r) => r["room"] != "")
        |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
        |> yield(name: "temperature_data")
    '''
    result = query_api.query_data_frame(query)
    return result

def load_heating_data(start_date, end_date):
    """Load heating valve/actuator data from InfluxDB"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "heating")
        |> filter(fn: (r) => r["_field"] =~ /valve_position|heating_demand|setpoint/)
        |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
        |> yield(name: "heating_data")
    '''
    result = query_api.query_data_frame(query)
    return result

def load_energy_data(start_date, end_date):
    """Load heating energy consumption data"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "energy")
        |> filter(fn: (r) => r["_field"] =~ /heating_consumption|heat_pump_power/)
        |> aggregateWindow(every: 5m, fn: mean, createEmpty: false)
        |> yield(name: "energy_data")
    '''
    result = query_api.query_data_frame(query)
    return result

## 4. Load and Prepare Data

In [ ]:
# Define analysis period
end_date = datetime.now(pytz.UTC)
start_date = end_date - timedelta(days=7)

# Load data
print(f"Loading data from {start_date} to {end_date}")
temp_data = load_temperature_data(start_date.isoformat(), end_date.isoformat())
heating_data = load_heating_data(start_date.isoformat(), end_date.isoformat())
energy_data = load_energy_data(start_date.isoformat(), end_date.isoformat())

print(f"Temperature data shape: {temp_data.shape}")
print(f"Heating data shape: {heating_data.shape}")
print(f"Energy data shape: {energy_data.shape}")

# Get list of rooms
rooms = temp_data['room'].unique() if 'room' in temp_data.columns else []
print(f"\nRooms found: {list(rooms)}")

## 5. Room Temperature Analysis

In [ ]:
# Analyze temperature patterns by room
if 'room' in temp_data.columns:
    # Create interactive plot with plotly
    fig = make_subplots(
        rows=len(rooms), cols=1,
        subplot_titles=[f'Temperature: {room}' for room in rooms],
        shared_xaxes=True
    )
    
    for i, room in enumerate(rooms, 1):
        room_data = temp_data[temp_data['room'] == room]
        fig.add_trace(
            go.Scatter(
                x=room_data['_time'],
                y=room_data['_value'],
                name=room,
                mode='lines'
            ),
            row=i, col=1
        )
    
    fig.update_layout(
        height=200*len(rooms),
        title_text="Room Temperature Patterns",
        showlegend=False
    )
    fig.update_xaxes(title_text="Time", row=len(rooms), col=1)
    fig.update_yaxes(title_text="Temperature (°C)")
    fig.show()
else:
    print("No room data available in temperature measurements")

## 6. Heating Schedule Analysis

In [ ]:
# Analyze heating schedules by hour of day and day of week
if not temp_data.empty:
    temp_data['hour'] = pd.to_datetime(temp_data['_time']).dt.hour
    temp_data['weekday'] = pd.to_datetime(temp_data['_time']).dt.day_name()
    temp_data['is_weekend'] = pd.to_datetime(temp_data['_time']).dt.weekday >= 5
    
    # Average temperature by hour for weekdays vs weekends
    hourly_patterns = temp_data.groupby(['hour', 'is_weekend', 'room'])['_value'].mean().reset_index()
    
    # Plot heating patterns
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, room in enumerate(rooms[:4]):  # Plot first 4 rooms
        room_patterns = hourly_patterns[hourly_patterns['room'] == room]
        weekday_data = room_patterns[~room_patterns['is_weekend']]
        weekend_data = room_patterns[room_patterns['is_weekend']]
        
        ax = axes[idx]
        ax.plot(weekday_data['hour'], weekday_data['_value'], 'b-', label='Weekday', linewidth=2)
        ax.plot(weekend_data['hour'], weekend_data['_value'], 'r-', label='Weekend', linewidth=2)
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Temperature (°C)')
        ax.set_title(f'{room} - Daily Temperature Pattern')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 23)
    
    plt.tight_layout()
    plt.show()

## 7. Setpoint vs Actual Temperature Analysis

In [ ]:
# Compare setpoints with actual temperatures
# This section would merge temperature and setpoint data

# Placeholder for analysis
print("Setpoint vs Actual Temperature Analysis")
print("=======================================")
print("This analysis would show:")
print("- How well rooms maintain their setpoint temperatures")
print("- Average deviation from setpoint by room")
print("- Time to reach setpoint after changes")
print("- Overshoot/undershoot patterns")

# Example visualization structure
fig, ax = plt.subplots(figsize=(12, 8))
# Placeholder scatter plot
ax.set_xlabel('Setpoint Temperature (°C)')
ax.set_ylabel('Actual Temperature (°C)')
ax.set_title('Setpoint vs Actual Temperature')
ax.plot([18, 24], [18, 24], 'r--', label='Perfect Match')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 8. Energy Consumption Analysis

In [ ]:
# Analyze heating energy consumption patterns
if not energy_data.empty:
    energy_data['hour'] = pd.to_datetime(energy_data['_time']).dt.hour
    energy_data['date'] = pd.to_datetime(energy_data['_time']).dt.date
    
    # Daily energy consumption
    daily_consumption = energy_data.groupby('date')['_value'].sum() * 5 / 60  # Convert to kWh
    
    # Hourly consumption pattern
    hourly_consumption = energy_data.groupby('hour')['_value'].mean()
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Daily consumption
    ax1.bar(daily_consumption.index, daily_consumption.values, color='orange')
    ax1.set_ylabel('Energy Consumption (kWh)')
    ax1.set_title('Daily Heating Energy Consumption')
    ax1.grid(True, alpha=0.3)
    
    # Hourly pattern
    ax2.plot(hourly_consumption.index, hourly_consumption.values, 'b-', linewidth=2)
    ax2.fill_between(hourly_consumption.index, hourly_consumption.values, alpha=0.3)
    ax2.set_xlabel('Hour of Day')
    ax2.set_ylabel('Average Power (W)')
    ax2.set_title('Average Hourly Heating Power Consumption')
    ax2.set_xlim(0, 23)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate statistics
    print(f"\nHeating Energy Statistics:")
    print(f"Average daily consumption: {daily_consumption.mean():.2f} kWh")
    print(f"Peak daily consumption: {daily_consumption.max():.2f} kWh")
    print(f"Total consumption (period): {daily_consumption.sum():.2f} kWh")

## 9. Room Efficiency Analysis

In [ ]:
# Analyze heating efficiency by room
# This would calculate metrics like:
# - Temperature stability (standard deviation)
# - Response time to heating
# - Energy per degree of heating

if 'room' in temp_data.columns:
    room_stats = temp_data.groupby('room')['_value'].agg(['mean', 'std', 'min', 'max'])
    room_stats['range'] = room_stats['max'] - room_stats['min']
    room_stats = room_stats.round(2)
    
    print("Room Temperature Statistics")
    print("==========================")
    print(room_stats)
    
    # Visualize room efficiency
    fig, ax = plt.subplots(figsize=(10, 6))
    room_stats['std'].plot(kind='bar', ax=ax, color='skyblue')
    ax.set_xlabel('Room')
    ax.set_ylabel('Temperature Std Dev (°C)')
    ax.set_title('Temperature Stability by Room (Lower is Better)')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 10. Optimization Opportunities

In [ ]:
# Identify optimization opportunities
print("Heating Optimization Opportunities")
print("=================================")

# 1. Identify overheated periods
if not temp_data.empty:
    overheated = temp_data[temp_data['_value'] > 23]  # Assuming 23°C as upper comfort limit
    if not overheated.empty:
        overheated_hours = overheated.groupby('hour').size()
        print(f"\n1. Overheating detected during hours: {list(overheated_hours.index)}")
    
    # 2. Identify rooms with high variability
    if 'room' in temp_data.columns:
        high_variability = room_stats[room_stats['std'] > 1.0]
        if not high_variability.empty:
            print(f"\n2. Rooms with high temperature variability: {list(high_variability.index)}")
    
    # 3. Identify potential schedule optimizations
    night_temps = temp_data[temp_data['hour'].between(0, 6)]
    avg_night_temp = night_temps.groupby('room')['_value'].mean()
    high_night_temp = avg_night_temp[avg_night_temp > 20]
    if not high_night_temp.empty:
        print(f"\n3. Rooms with high night temperatures: {list(high_night_temp.index)}")
        print("   Consider reducing night setpoints for energy savings")

# 4. Calculate potential savings
print("\n4. Potential Energy Savings:")
print("   - 1°C reduction in average temperature: ~6% energy savings")
print("   - Optimized scheduling: 10-15% potential savings")
print("   - Zone-based control: 5-10% additional savings")

## 11. Key Findings and Recommendations

### Summary

Based on the heating pattern analysis, document:

1. **Current Heating Patterns**
   - Peak heating hours
   - Room-specific schedules
   - Weekend vs weekday differences

2. **Energy Consumption**
   - Daily consumption trends
   - High consumption periods
   - Room efficiency rankings

3. **Optimization Opportunities**
   - Schedule adjustments
   - Temperature setpoint optimization
   - Zone control improvements

### Recommendations

1. **Immediate Actions**
   - Reduce night-time setpoints in unoccupied rooms
   - Adjust heating schedules based on actual occupancy
   - Fix rooms with poor temperature stability

2. **Medium-term Improvements**
   - Implement predictive heating based on weather forecasts
   - Optimize heat pump operation for efficiency
   - Consider zone-based control strategies

3. **Long-term Strategies**
   - Upgrade insulation in rooms with high heat loss
   - Install smart thermostatic valves
   - Integrate with renewable energy production